# Enoncé du Projet

* Il s'agit d'implémenter $5$ modèles pour un système de recommendation de films à partir des données Netflix. 

* Les modèles à implémenter sont présentés dans l'[article](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf) de Koren. Le dernier modèle de l'article est intitulé **Integrated Model**. Celui-ci a remporté le [Prix Netflix](https://en.wikipedia.org/wiki/Netflix_Prize) en $2008$. 

* Les données (téléchargeables à partir de ce [lien](https://archive.org/download/nf_prize_dataset.tar))  contiennent $100$ millions de notations (ratings) de $480 000$ utilisateurs sur $17 000$ films. Le but de chacun des modèles à implémenter consiste à **prédire** la notation d'un **utilisateur** donné sur l'ensemble des **films** que celui-ci n'a pas encore regardés. La recommendation pour cet utilisateur consiste à lui proposer la liste des films par ordre décroissant de notations prédites.

* Chaque modèle est formulé comme une [Régression Linéaire](https://colab.research.google.com/drive/13tj_epwjFWkxBrcXwm0EELh0a81K1ID2#scrollTo=F075W1a2BWSe) dont les paramètres sont à estimer par procédure de [Gradient Descent](https://colab.research.google.com/drive/13tj_epwjFWkxBrcXwm0EELh0a81K1ID2#scrollTo=VqJhJGBduTXw). La performance de chaque modèle est évaluée par le critère RMSE (racine carrée de la Mean Squared Error vue dans le  [Chapitre Régression](https://colab.research.google.com/drive/13tj_epwjFWkxBrcXwm0EELh0a81K1ID2#scrollTo=X8iC5mkh3orV)).

* Les modèles à implémenter sont:
  1. Baseline Estimates décrit dans la section $2.1$ de l'article;
  2. Correlation-Based Neighbourhood décrit par l'équation $(3)$ dans la section $2.2$;
  3. Correlation-Based Neighbourhood with implicit data décrit par l'équation $(10)$ dans la section $3$;
  4. SVD++ décrit par l'équation $(15)$ dans la section $4$;
  5. Integrated Model décrit par l'équation $(16)$ dans la section $5$. 

* Dans la méthodologie ci-dessous, nous décrivons la façon d'organiser les données ainsi que les $5$ modèles à implémenter. Les notations de l'article ont été conservées afin de faciliter le basculement entre le notebook et l'article.

* Le code final doit être livré en $6$ fichiers: le premier pour la compilation et des ensembles d'apprentissage et de tests; les $5$ pour chacun des modèles à implémenter. Dans le cas d'utilisation de librairies externes, mettre en commentaire la procédure d'installation de celle-ci dans chaque fichier où elle est utilisée.

* Pour toute question ou incompréhension vous êtes invités à  m'écrire sur mon adresse mail: nedjmeddine.allab@gmail.com ou via  messagerie sur [linkedIn](https://www.linkedin.com/in/nedjmeddine-allab-phd-87201243/) pour des échanges plus rapides.

* Bon courage.




# Methodology

## 1. Data Compilation

### 1.1. Generation of the rating dataset

* Write Python function **rating_compiler** to compile from the training_set.tar, the $17770$ files and store the result into one data structure named $\bf{D}$;

* $\bf{D}$ can be a numpy matrix, csv file/pandas DataFrame, sparse matrix, hdf5 file...;

### 1.2. Generation of training and test datasets

* Extract from $\bf{D}$ the ratings corresponding to the users and movies described in the probe.txt and store the result into one data structure named $\bf{T}$;

* Construct training dataset $\bf{R}$ as $\bf{D}$ from which we remove entries present in $\bf{T}$;  

* $\bf{D}$, $\bf{T}$ and $\bf{R}$ must all have the same format (for example users as rows and films as columns);

## 2. Baseline Estimates

* Let $u$ and $v$ be two users and $i$ and $j$ two films;

* we define:
  *  $r_{ui}$ as the rating by user $u$ on film $i$;
  *  $\hat{r}_{ui}$ as the **predicted** rating of $r_{ui}$;


* In Netflix data $99\%$ of ratings are missing, the $(u,i)$ pairs for which $r_{ui}$ is known are stored in the set

\begin{equation}
\mathcal{K} = \{(u,i) \quad \vert \quad   r_{ui} \quad \mbox{is known}\}.
\end{equation}

\\

* In rating data, we tend to have users who systematically give higher ratings than others and also, some movies which receive higher ratings than others;

* In Section 2.1 of the [article](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf) these tendencies are considered as baseline ratings $b_{ui}$ and are defined as

\begin{equation}
b_{ui} = \mu + b_u + b_i,
\end{equation}

\\

* where:
  *  $\mu$ is the overall average rating;
  * $b_u$ the observed deviation of user $u$;
  * $b_i$ the observed deviation of movie $i$;

\\



* Estimates of $b_u$s and $b_i$s are obtained by the minimization of the regularized MSE loss function

\begin{equation}
\sum_{(u,i) \in \mathcal{K}} (r_{ui} - b_{ui})^2 + \lambda_1 \left( \sum_{u} b_u^2 + \sum_{i} b_i^2 \right),
\end{equation}

\\

  - where $\lambda_1 \left( \sum_{u} b_u^2 + \sum_{i} b_i^2 \right)$ is the regularization term to avoid overfitting. The penality coefficient $\lambda_1 = 0.02$.

\\

* Write a python function **baseline_estimator** to estimate $b_u, b_i$ for every $(u,i) \in \mathcal{K}$;

\\

* The problem above can easily be transformed into linear regression problem and the minimization of the regularized MSE can be done through gradient descent.  

## 3. Neighbourhood Models

### 3.1 Correlation-Based Neighbourhood Model

* To predict $r_{ui}$ neighbourhood approach look for a list of $k$ movies $\{j_1,\ldots,j_k \}$ such that:
  * the movies are already rated by $u$;
  * the movies are appreciated by the same group of users suggesting their similarity with movie $i$;  

* Similarity between movies $i$ and $j$ is defined as $s_{ij}$ and it is based on the pearson coefficient of correlation $\rho_{ij}$   

\\

\begin{equation}
s_{ij} = \frac{n_{ij}}{n_{ij} + \lambda_2} \rho_{ij},
\end{equation}

\\

* where $n_{ij}$ is the number of users who rated both movies $i$ and $j$, $\lambda_2$ is a regulation coefficient typically equal to $100$.

\\

* To predict $r_{ui}$:
  * define the set $S^k (i;u)$ including the $k$ movies rated by $u$ and the most similar to $i$ according to $s_{ij}$;
  * estimate $r_{ui}$ as the weighted sum of $r_{uj}$ such that

\begin{equation}
\hat{r}_{ui} = b_{ui} + \frac{1}{\sum_{j \in S^k (i;u)} s_{ij}} \sum_{j \in S^k (i;u)} s_{ij} (r_{uj} - b_{uj}).
\end{equation}

\\

* Write a Python function **correlation_based_neighbourhood_model** that estimate $r_{ui}$ according to the above equation. 
  

### Correlation-Based Neighbourhood Model with Implicit Feedback

* In Section 2.5 of [article](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf), the author proposes to predict rating $r_{ui}$ from ***explicit*** neighbouring ratings $r_{uj}$ but also from ***implicit*** data such that: movie rental history or visited movies in the graphical interface;

* Explicit and implicit data are formalized as follows: 

  * Let $u$ be a user associated with:
    * $\mbox{R}(u)$: set of movies for which ratings by $u$ are available;
    * $\mbox{N}(u)$: set of movies for which $u$ provided an implicit preference;

* In the case of Netflix Data, implicit data is inferred from *which* movies have been rated. This implicitly tells us about preferences of the user;

* Implicit feedback are derrived from ratings as binary matrix where:
  * implicit preference of user $u$ to movie $i$ equals to 1 if $r_{ui}$ exists;
  * implicit preference of user $u$ to movie $i$ equals to 0 if $r_{ui}$ does not exist;

* The new model will be developped from the the following prediction rule

\\

\begin{equation}
\hat{r}_{ui} = b_{ui} + \sum_{j \in \mbox{R}(u)} (r_{uj} - b_{uj}) w_{ij},
\end{equation}

* where $w_{ij}$ are associated to the similarity between movies $i$ and $j$.

\\

* Note that here the author considers $(r_{uj} - b_{uj})$ as the weight to be applied on $w_{ij}$ such that:

  * $w_{ij}$ is high if movies $i$ and $j$ are related and low otherwise;
  * in case $w_{ij}$ is high and user $u$ rated $j$ higher than expected then $(r_{uj} - b_{uj})$ will be greater and thus the contribution of $w_{ij}$ will be significant;
  * if however $w_{ij}$ is high but user $u$ rated $j$ just as expected leading to low  $(r_{uj} - b_{uj})$ the contribution of $w_{ij}$ will not be significant;
  * This mecanism is interesting because it gives weight only to the movies $j$ related to $i$ that user $u$ really enjoyed. 

* Note also that in the equation above we consider **all** movies for which user $u$ has given an **explicit** rating;

* To include **implicit** feedback the author adds to the equation above another weights $c_{ij}$  

\\

\begin{equation}
\hat{r}_{ui} = b_{ui} + \sum_{j \in \mbox{R}(u)} (r_{uj} - b_{uj}) w_{ij} + \sum_{j \in \mbox{N}(u)} c_{ij},
\end{equation}

\\

* where $c_{ij}$ is high if movie $j$ is related to $i$ and if user $u$ implicitly said that $j$ belongs to his category of films.

\\

* The author suggests to normalize the equation above by the suqared root of $\mbox{R}(u)$ and $\mbox{N}(u)$ sizes   

\\

\begin{equation}
\hat{r}_{ui} = b_{ui} + \frac{1}{\sqrt{|\mbox{R}(u)|}}\sum_{j \in \mbox{R}(u)} (r_{uj} - b_{uj}) w_{ij} + \frac{1}{\sqrt{|\mbox{N}(u)|}}\sum_{j \in \mbox{N}(u)} c_{ij},
\end{equation}

\\

* For a better computation efficiency, only the $k$ nearest movies will be selected from explicit and implicit sets and respectively stored in $\mbox{R}^k (i;u)$ and $\mbox{N}^k (i;u)$.

\\

* Finally, the prediction rule is defined as follows

\\

\begin{equation}
\hat{r}_{ui} = \mu + b_u + b_i + \frac{1}{\sqrt(|\mbox{R}^k (i;u)|)}\sum_{j \in \mbox{R}^k (i;u)} (r_{uj} - b_{uj}) w_{ij} + \frac{1}{\sqrt(|\mbox{N}^k (i;u)|)}\sum_{j \in \mbox{N}^k (i;u)} c_{ij}.
\end{equation}

\\

* The $b_{uj}$ of the $4^{th}$ term are the baseline prediction estimated in your first function. In this new model they are considered as constants;

* The parameters are: $b_u, b_i, w_{ij}, c_{ij}$ estimated by the minimization of the regularized MSE loss function

\\

\begin{equation}
\sum_{(u,i) \in \mathcal{K}} (r_{ui} - \hat{r}_{ui})^2 + \lambda_4 \left( \sum_{u} b_u^2 + \sum_{i} b_i^2  + \sum_{j \in \mbox{R}^k (i;u)} w_{i,j}^2 + \sum_{j \in \mbox{N}^k (i;u)} c_{i,j}^2 \right).
\end{equation}

\\

* The author suggests regularization coefficient $\lambda_4 = 0.002$.

* Write a Python function **correlation_based_implicit_neighbourhood_model** that estimate $r_{ui}$ according to the above equation. 




## 4. Latent Factor Models

#### Singular Value Decomposition


* Latent factor models explain observed ratings as a dot product between two latent (or feature) vectors $p_u$ and $q_i$ both defined in  ${\rm I\!R}^f$;

* In the context of recommender systems, $p_u$ may describe user $u$ in terms of his belonging to $f$ user categories while $q_i$, may represent for the movie $i$ the proportions of $f$ genres that it contains;

* Singular Value Decomposition ([SVD](https://colab.research.google.com/drive/1xJHWm1LS39u4NDxnRsfjaxrux8HE1nbg#scrollTo=rgGYLnWH42aC)) of user-movie rating matrix is widely used and lead to the following prediction model  

\\

\begin{equation}
\hat{r}_{ui} = b_{ui} + p_u^T q_i.
\end{equation}

\\

* In practice, SVD raises difficulties due to the high portion of missing ratings ($99\%$ in the Netflix Dataset), we tend to model only the observed ratings by minimizing the regularized MSE loss function

\\

\begin{equation}
\sum_{(u,i) \in \mathcal{K}} (r_{ui} - \hat{r}_{ui})^2 + \lambda_3 \left( \sum_{u} b_u^2 + \sum_{i} b_i^2  + ||p_u||^2 + ||q_i||^2 \right).
\end{equation}

### Improved Singular Value Decomposition

* **Patereck** proposes in his [article](https://www.cs.uic.edu/~liub/KDD-cup-2007/proceedings/Regular-Paterek.pdf) to model $p_u$ only on rated movies;

* In that case, each movie $i$ becomes associated with **two** factors vectors $q_i$ and $x_i$ such that each user $u$ will be represented by the factor vector 

\\

\begin{equation}
p_u = \frac{1}{\sqrt{ | \mbox{R}(u) |}} \sum_{j \in \mbox{R}(u)} x_j,
\end{equation}

\\

* leading to the new prediction model

\\

\begin{equation}
\hat{r}_{ui} = b_{ui} + q_i^T \frac{1}{\sqrt{ | \mbox{R}(u) |}} \sum_{j \in \mbox{R}(u)} x_j.
\end{equation}

### Asymmetric SVD

* Following the improved SVD paradigm, **Koren** came up with asymmetric SVD that includes implicit feedback;

* Each movie $i$ is associated with $3$ factors vectors $q_i, x_i, y_i \in {\rm I\!R}^f$ leading to the following prediction model

\\

\begin{equation}
\hat{r}_{ui} = b_{ui} + q_i^T \left( \frac{1}{\sqrt{ | \mbox{R}(u) |}} \sum_{j \in \mbox{R}(u)} (r_{uj} - b_{uj}) x_j +  \frac{1}{\sqrt{ | \mbox{N}(u) |}}\sum_{j \in \mbox{N}(u)} y_j \right).
\end{equation}

\\

* This model presents many advantages such as:
  * new users do not require retraining as it depends only on movies;
  * recommendations can be explained in terms of movies only;
  * it has less parameters when the number of users is larger than the number of films;

\\

* Just like the other models, parameters are estimated through the minimization of regularized MSE loss function.

### SVD++

* **Koren** highlighted in Section 4 of his [article](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf) that introducing implicit feedback directly in the SVD prediction rule will yield better results

\\

\begin{equation}
\hat{r}_{ui} = b_{ui} + q_i^T \left( p_u  +  \frac{1}{\sqrt{ | \mbox{N}(u) |}}\sum_{j \in \mbox{N}(u)} y_j \right).
\end{equation}

\\

* Write a Python program named **SVD++** that estimate $r_{ui}$ according to the above equation. Estimation of parameters is done through the minimization of a regularized MSE Loss function.


## 5. Integrated Model

* Empiric research shows that Neighborhood
and factor models do not capture same informations; 

* **Koren** ultimately proposed in Section 5 of his [article](https://www.cs.rochester.edu/twiki/pub/Main/HarpSeminar/Factorization_Meets_the_Neighborhood-_a_Multifaceted_Collaborative_Filtering_Model.pdf) to combine between Latent and Neighbourhood models to enrich each other as 

\\

\begin{equation}
\hat{r}_{ui} = \mu + b_u + b_i + q_i^T \left( p_u  +  \frac{1}{\sqrt{ | \mbox{N}(u) |}}\sum_{j \in \mbox{N}(u)} y_j \right) + \frac{1}{\sqrt{|\mbox{R}(u)|}}\sum_{j \in \mbox{R}(u)} (r_{uj} - b_{uj}) w_{ij} + \frac{1}{\sqrt{|\mbox{N}(u)|}}\sum_{j \in \mbox{N}(u)} c_{ij}.
\end{equation}

\\

* First three terms describe general properties of the movie and the user;

* Forth term provides the interaction between user's profile and  movie's profile;

* Last two terms contributes fine grained adjustments that are hard to profile;

* Write Python program named **integrated_model** that estimate $r_{ui}$ according to the above equation by minimizing regularized MSE Loss function.